# Deep Convnets

In [10]:
import keras
from keras import models, layers
import numpy as np
import matplotlib.pyplot as plt
from kapre.time_frequency import Melspectrogram
from kapre.utils import Normalization2D
import sys
sys.path.append('../')
from utils import DataFeed
from keras import regularizers
from keras.optimizers import RMSprop, Nadam
from keras.preprocessing.image import ImageDataGenerator

### Load Data

In [11]:
data_path='../preprocessing/preprocessed_data'

In [12]:
train_data, train_labels = DataFeed.Dataset.create(data_path, ['train/voxforge', 'train/youtube'], num=50000, use_premade=True)
val_data, val_labels = DataFeed.Dataset.create(data_path, ['val/youtube', 'val/voxforge'], num=-1, shuffle=True)

In [13]:
callback_stopearly = keras.callbacks.EarlyStopping(monitor='val_acc',
                                                   patience=3)

## Topcoder_net
based on [repo](https://github.com/pietz/language-recognition/blob/master/Language%20Classifier.ipynb)

In [24]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=192, n_hop=418,
                         fmin=0.0, fmax=8000, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
model.add(layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.BatchNormalization())

model.add(layers.SeparableConv2D(64, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.BatchNormalization())

model.add(layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D())
model.add(layers.BatchNormalization())

model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(3, activation='softmax'))
#model.summary()

=> up to xxx%

## Resnet

In [14]:
from keras.applications import vgg16, resnet50
from keras_contrib.applications.resnet import ResNet18, ResNet

In [41]:
datagen = ImageDataGenerator(
                            featurewise_center=True,
                            featurewise_std_normalization=True,
                            rotation_range=20,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            horizontal_flip=True,
                            dtype='float32')

In [43]:
model = models.Sequential()
model.add(Melspectrogram(n_dft=512, input_shape=(1, 5 * 16000,),
                         padding='same', sr=16000, n_mels=192, n_hop=418,
                         fmin=0.0, fmax=8000, power_melgram=1.0,
                         return_decibel_melgram=False, trainable_fb=False,
                         trainable_kernel=False))
#model.add(layers.Lambda(baum)) try using data augmentation
model.add(ResNet(input_shape=(192, 192, 1), classes=3, block='basic', repetitions=[2, 2, 2], dropout=0.5))
model.summary()

reshaping via a convolution...
reshaping via a convolution...
reshaping via a convolution...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
melspectrogram_18 (Melspectr (None, 192, 192, 1)       312512    
_________________________________________________________________
model_8 (Model)              (None, 3)                 2788419   
Total params: 3,100,931
Trainable params: 2,783,811
Non-trainable params: 317,120
_________________________________________________________________


=> up to 76% val_acc, overfits

### Train Model

In [8]:
model.compile(optimizer=RMSprop(lr=0.001, decay=1e-8),
              metrics=['accuracy'],
              loss='categorical_crossentropy')

In [ ]:
model.fit(train_data, train_labels, 
          epochs=2,
          batch_size=64,
          shuffle=True,
          validation_data=(val_data, val_labels))

Train on 50000 samples, validate on 10000 samples
Epoch 1/2
49792/50000 [============================>.] - ETA: 0s - loss: 0.7944 - acc: 0.6447